In [1]:
import copy
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import utils

import warnings; warnings.simplefilter('ignore')

from hmmlearn import hmm
from mpl_toolkits.mplot3d import Axes3D
from nilearn import datasets, plotting
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Low Dimensional Connectome Dynamics

### Prepare Data - Separate Modalities

In [2]:
all_subjects_all_trials_connectomes = utils.load_connectomes(utils.ALL_SUBJECT_IDS, utils.ALL_TRIAL_IDS)
all_subjects_all_trials_connectomes['fmri'].shape

(17906, 68, 68)

Extract flattened representation of upper triangular of Pearson correlation matrix for each connectome type.

In [3]:
# NOTE: The below logic would have to change if we move away from using Desikan Atlas where the number of regions 
# are the same between EEG and fMRI
num_regions = all_subjects_all_trials_connectomes['fmri'].shape[1]
num_regions

68

In [4]:
upper_triangular_including_diagonal_idxs = np.triu_indices(num_regions, k=0)
lower_triangular_idxs = np.tril_indices(num_regions, k=-1)

In [5]:
all_subjects_all_trials_connectome_upper_triangular_flattened = copy.deepcopy(all_subjects_all_trials_connectomes)
for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    all_subjects_all_trials_connectome_upper_triangular_flattened[k] = np.array([c[upper_triangular_including_diagonal_idxs].flatten() for c in all_subjects_all_trials_connectomes[k]])

In [ ]:
all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape

### Prepare Data - Combined Modalities

Combine all modalities into a single time series by smushing feature vectors together.

In [7]:
num_features_per_modality = all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape[1]
fmri_feature_idxs  = range(num_features_per_modality*0, num_features_per_modality*1)
alpha_feature_idxs = range(num_features_per_modality*1, num_features_per_modality*2)
beta_feature_idxs  = range(num_features_per_modality*2, num_features_per_modality*3)
delta_feature_idxs = range(num_features_per_modality*3, num_features_per_modality*4)
gamma_feature_idxs = range(num_features_per_modality*4, num_features_per_modality*5)
theta_feature_idxs = range(num_features_per_modality*5, num_features_per_modality*6)

feature_idxs = [('fmri', fmri_feature_idxs),
                ('alpha', alpha_feature_idxs),
                ('beta', beta_feature_idxs),
                ('delta', delta_feature_idxs),
                ('gamma', gamma_feature_idxs),
                ('theta', theta_feature_idxs)]

In [8]:
all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened = []
for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened.append(all_subjects_all_trials_connectome_upper_triangular_flattened[k])
all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened = np.concatenate(all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened, axis=1)

In [9]:
all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened.shape

(17906, 14076)

In [10]:
all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened[:, fmri_feature_idxs].shape

(17906, 2346)

## Hidden Markov Models

### Training

(1) TODO: BIC and AIC as a normalization scheme

(2) TODO: PCA first then project back out through PCs

(3) TODO: Train HMMs on subset of data and predict likelihood of new data

(4) TODO: Find a bunch of graph statistics (node-wise or global) and compute the time series of the graph statistics and fit an HMM to that.

In [ ]:
# def train_optimal_hmm_on_data(data, bic=True, pca_variance_retained=0.99, forced_component_count=None):
    
#     # Apply PCA to dimensionality reduce the data and retain xx% variance
#     print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
#     pca_model = PCA(pca_variance_retained)
#     dim_reduced_data = pca_model.fit_transform(data)
#     hmm_preprocessing_pca_models.append(pca_model)
#     print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    
#     # Find HMM model with lowest BIC/AIC score with binary search
#     bottom_n_components = 2
#     top_n_components = min(100, dim_reduced_data.shape[0]-1)
    
#     while bottom_n_components != top_n_components:
        
#         # Select mid of component range
#         print("\t\t Searching for num components in hyperparameter range: {0}-{1}".format(bottom_n_components, top_n_components))
#         n_components = (top_n_components+bottom_n_components)//2
        
#         # Check end condition
#         if bottom_n_components == top_n_components:
#             print("\t\tFound optimal number of components: {0}".format(n_components))
#             return hmm.GaussianHMM(n_components=n_components,
#                                    covariance_type="diag").fit(dim_reduced_data)

#         # Train HMM model
#         print("\t\t training hmm with n_components={0}".format(n_components))
#         candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
#                                               covariance_type="diag").fit(dim_reduced_data)
#         print("\t\t training hmm with n_components={0}".format(n_components-1))
#         candidate_hmm_model_1 = hmm.GaussianHMM(n_components=n_components-1,
#                                                covariance_type="diag").fit(dim_reduced_data)
#         # Compute BIC/AIC score
#         n = dim_reduced_data.shape[0]
#         k = num_parameters_in_hmm(candidate_hmm_model)
#         L = candidate_hmm_model.decode(dim_reduced_data)[0]
#         score = (np.log(n)*k - 2*L) if bic else (k - L)

#         print("\t\t\tn={0}, k={1}, L={2}".format(n, k, L))
#         print("\t\t\tScore={0}".format(score))
        
#         k_1 = num_parameters_in_hmm(candidate_hmm_model_1)
#         L_1 = candidate_hmm_model_1.decode(dim_reduced_data)[0]
#         score_1 = (np.log(n)*k_1 - 2*L_1) if bic else (k_1 - L_1)

#         print("\t\t\tn={0}, k_1={1}, L_1={2}".format(n, k_1, L_1))
#         print("\t\t\tScore_1={0}".format(score_1))
    
#         if score_1 <= score:
#             top_n_components = n_components
#         else:
#             bottom_n_components = n_components

#     return None

In [37]:
def num_parameters_in_hmm(model):  
    k = model.means_.shape[0]
    d = model.means_.shape[1]
    # NOTE: We are using a diagonal covariance matrix
    return (k*d)+(k*d)+k+(k**2)

In [42]:
def train_optimal_hmm_on_data(data, bic=True, pca_variance_retained=0.99, forced_component_count=None):
    
    # Apply PCA to dimensionality reduce the data and retain xx% variance
    print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
    pca_model = PCA(pca_variance_retained)
    dim_reduced_data = pca_model.fit_transform(data)
    hmm_preprocessing_pca_models.append(pca_model)
    print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    
    # Compute BIC/AIC scores for all hmm models with num_states between 2 and 50
    scores = []
    candidate_hmm_models = []
    
    for n_components in (range(1, data.shape[0]-1) if not forced_component_count else range(forced_component_count, forced_component_count+1)):
        
        print("\t\t training hmm with n_components={0}".format(n_components))

        # Train HMM model
        candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
                                              covariance_type="diag").fit(dim_reduced_data)
        candidate_hmm_models.append(candidate_hmm_model)

        # Compute BIC/AIC score
        n = dim_reduced_data.shape[0]
        k = num_parameters_in_hmm(candidate_hmm_model)
        L = candidate_hmm_model.decode(dim_reduced_data)[0]

        print("\t\t\tn={0}, k={1}, L={2}".format(n, k, L))

        score = (np.log(n)*k - 2*L) if bic else (k - L)
        scores.append(score)
        print("\t\t\tScore={0}".format(score))
        
        if len(scores) >= 3:
            # Last two increases in number of components yielded worse scores -> exit search
            if scores[-1] > scores[-2] and scores[-2] > scores[-3]:
                break
    
    # Select the model with the lowest BIC/AIC score
    selected_hmm_model = candidate_hmm_models[np.argmin(scores)]
    return selected_hmm_model, pca_model

Train HMMs separately on data from each modality using the Bayesian Information Criterion (BIC) to select the number of components.

In [43]:
hmm_preprocessing_pca_models = []
hmm_models = []

for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    
    print(k)
    data = all_subjects_all_trials_connectome_upper_triangular_flattened[k]
    hmm_model, pca_model = train_optimal_hmm_on_data(data, pca_variance_retained=0.90)
    hmm_models.append(hmm_model)
    hmm_preprocessing_pca_models.append(pca_model)
    

fmri
	Applying pca and retaining 90.00% variance...
	2346 dimensions -> 470 dimensions
		 training hmm with n_components=1
			n=17906, k=942, L=-7513945.386984726
			Score=15037115.677415038
		 training hmm with n_components=2
			n=17906, k=1886, L=-7458867.1515939785
			Score=14936203.69586139
		 training hmm with n_components=3
			n=17906, k=2832, L=-7439867.45385991
			Score=14907468.375403363
		 training hmm with n_components=4
			n=17906, k=3780, L=-7426174.119843681
			Score=14889365.368163276
		 training hmm with n_components=5
			n=17906, k=4730, L=-7414373.305099972
			Score=14875066.985250492
		 training hmm with n_components=6
			n=17906, k=5682, L=-7404173.5841353405
			Score=14863990.375678126
		 training hmm with n_components=7
			n=17906, k=6636, L=-7391452.563198404
			Score=14847890.751943413
		 training hmm with n_components=8
			n=17906, k=7592, L=-7378460.476156975
			Score=14831268.581781976
		 training hmm with n_components=9
			n=17906, k=8550, L=-7364270.1241061

			n=17906, k=78386, L=-6527161.449187555
			Score=13821948.462584892
		 training hmm with n_components=78
			n=17906, k=79482, L=-6496599.387513669
			Score=13771557.347916909
		 training hmm with n_components=79
			n=17906, k=80580, L=-6500294.509535411
			Score=13789700.186422443
		 training hmm with n_components=80
			n=17906, k=81680, L=-6482192.4563910635
			Score=13764268.26037806
		 training hmm with n_components=81
			n=17906, k=82782, L=-6454319.799109482
			Score=13719314.711841475
		 training hmm with n_components=82
			n=17906, k=83886, L=-6455111.952175117
			Score=13731710.369781584
		 training hmm with n_components=83
			n=17906, k=84992, L=-6448903.122048723
			Score=13730123.647119895
		 training hmm with n_components=84
			n=17906, k=86100, L=-6436014.463989821
			Score=13715196.854375456
		 training hmm with n_components=85
			n=17906, k=87210, L=-6419530.779313029
			Score=13693099.594177496
		 training hmm with n_components=86
			n=17906, k=88322, L=-6405370.53616

KeyboardInterrupt: 

Train a single HMM on data from all modalities combined.

In [25]:
combined_modality_hmm_model, combined_modality_preprocessing_pca_model = train_optimal_hmm_on_data(all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened, pca_variance_retained=0.90)

	Applying pca and retaining 90.00% variance...
	14076 dimensions -> 2096 dimensions
		 training hmm with n_components=10
			n=17906, k=43953230, L=2382718.800805331
			Score=425663758.65272385


### Spatial Representation of Hidden States

SEPARATE MODALTIES - Plot spatial representation of each hidden markov model state in connectome space.

In [26]:
hmm_models[0].means_.shape

(10, 470)

In [27]:
hmm_preprocessing_pca_models[0].components_.shape

(470, 2346)

In [28]:
combined_modality_hmm_model, combined_modality_preprocessing_pca_model = combined_modality_hmm_model

In [ ]:
fig = plt.figure(figsize=(180, 60))
fig.suptitle('Spatial Loadings of Hidden Markov Model States', fontsize=40)

subplot_idx = 1
for (k, hmm_model, pca_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models, hmm_preprocessing_pca_models):
    for component_idx in range(0, n_components):
        
        # Extract connectome representation of the hidden state
        hidden_state = np.zeros((num_regions, num_regions))
        hidden_state[upper_triangular_including_diagonal_idxs] = hmm_model.means_[component_idx]
        hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

        # Plot connectome representation of the hidden state
        ax = fig.add_subplot(len(hmm_models), n_components, subplot_idx)
        plotting.plot_connectome(hidden_state, desikan_atlas_coordinates(), title='{0} HiddenState-{1} Connectome'.format(k, component_idx+1),
                                 edge_threshold='95%', node_size=20, colorbar=True, axes=ax)
        subplot_idx += 1

plt.savefig('output/hmm/spatial_loadings.png')
plt.close()

COMBINED MODALITIES - Plot spatial representation of each hidden markov model state in connectome space.

In [ ]:
combined_modality_hmm_model.means_.shape

In [ ]:
fig = plt.figure(figsize=(180, 60))
fig.suptitle('Spatial Loadings of Hidden Markov Model States', fontsize=40)

subplot_idx = 1
for feature_desc, feature_idx in feature_idxs:
    for component_idx in range(0, n_components):
        
        # Extract connectome representation of the hidden state
        hidden_state = np.zeros((num_regions, num_regions))
        hidden_state[upper_triangular_including_diagonal_idxs] = combined_modality_hmm_model.means_[component_idx][feature_idx]
        hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

        # Plot connectome representation of the hidden state
        ax = fig.add_subplot(len(feature_idxs), n_components, subplot_idx)
        plotting.plot_connectome(hidden_state, desikan_atlas_coordinates(), title='{0} HiddenState-{1} Connectome'.format(feature_desc, component_idx+1),
                                 edge_threshold='95%', node_size=20, colorbar=True, axes=ax)
        subplot_idx += 1

plt.savefig('output/hmm/combined_modality_spatial_loadings.png')
plt.close()

### Decoded Hidden State Sequence

SEPARATE MODALITIES - Compute decoded state sequences.

In [ ]:
decoded_state_sequences = []
decoded_state_sequences_log_likelihoods = []
for (k, hmm_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models):
    decoded_state_sequence = hmm_model.decode(all_subjects_all_trials_connectome_upper_triangular_flattened[k])
    decoded_state_sequences_log_likelihoods.append(decoded_state_sequence[0])
    decoded_state_sequences.append(decoded_state_sequence[1])
    
print(decoded_state_sequences_log_likelihoods)
print(sum(decoded_state_sequences_log_likelihoods)/len(decoded_state_sequences_log_likelihoods))

COMBINED MODALITIES - Compute decoded state sequence.

In [ ]:
combined_modality_log_likelihood, combined_modality_decoded_state_sequence = combined_modality_hmm_model.decode(all_modality_all_subjects_all_trials_connectome_upper_triangular_flattened)
print(combined_modality_log_likelihood/6)

SEPARATE MODALITIES - Compute decoded state sequence statistics.

In [ ]:
# Fractional Occupancy - the fraction of time spent in each state relative to the total duration
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Fractional Occupancy')
num_plots = len(decoded_state_sequences)
subplot_idx = 1

for (k, state_seq) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, decoded_state_sequences):
    
    # Compute fractional occupancy
    fractional_occupancies_per_state = []
    for state_idx in range(0, n_components):
        fractional_occupancy = len(state_seq[state_seq == state_idx])/len(state_seq)
        fractional_occupancies_per_state.append(fractional_occupancy)
        
    # Plot fractional occupancy per state
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.title(k)
    x = np.arange(n_components)
    plt.bar(x, height=fractional_occupancies_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
plt.savefig('output/hmm/fractional_occupancy.png')
fig.close()

In [ ]:
# Mean Life Time - the time spent in a state before transitioning to a new state on average
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Mean Life Time')
num_plots = len(decoded_state_sequences)
subplot_idx = 1

for (k, state_seq) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, decoded_state_sequences):

    # Compute mean life time per state
    mean_life_time_per_state = []
    for state_id in range(0, n_components):
        
        # Count number of transitions out of state with state_id
        num_transitions_out_of_state_with_state_id = 0
        for i in range(0, len(state_seq)-1):
            if state_seq[i] == state_id and state_seq[i+1] != state_id:
                num_transitions_out_of_state_with_state_id += 1
        
        # Count total number of time points spent in state with state id
        num_time_points_in_state_with_state_id = len(state_seq[state_seq == state_id])
        
        # Compute mean life time
        mean_life_time = num_time_points_in_state_with_state_id/num_transitions_out_of_state_with_state_id
        mean_life_time_per_state.append(mean_life_time)
    
    
    # Plot mean life time per state
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.title(k)
    x = np.arange(n_components)
    plt.bar(x, height=mean_life_time_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
plt.savefig('output/hmm/mean_life_time.png')
fig.close()

In [ ]:
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Transition Probabilities')
num_plots = len(hmm_models)
subplot_idx = 1

for (k, hmm_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models):
    
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.imshow(hmm_model.transmat_, cmap='gist_heat')
    plt.title(k)
    plt.colorbar()
    subplot_idx += 1

plt.savefig('output/hmm/transition_probabilities.png')
fig.close()

Plot transitions between decoded states as a time series.

In [ ]:
def get_n_colors(n):
    return [ cmx.rainbow(float(i)/n) for i in range(n) ]

In [ ]:
fig = plt.figure(figsize=(300, 25))
fig.suptitle('Decoded Hidden State Sequences w/ ' + str(n_components) + ' Components')

component_colors = get_n_colors(n_components)
num_plots = len(hmm_models)+1
subplot_idx = 1

fig.add_subplot(num_plots, 1, subplot_idx)
for i in range(0, n_components):
    plt.axvline(x=i, linewidth=1000, color=component_colors[i])
    x+=1
plt.title("Color Legend for each Hidden State")
plt.yticks([])
subplot_idx += 1

for (k, hmm_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models):
    
    print(k)
    decoded_state_sequence = hmm_model.decode(all_subjects_all_trials_connectome_upper_triangular_flattened[k])
    
    fig.add_subplot(num_plots, 1, subplot_idx)
    x = 0
    for state in decoded_state_sequence[1][:6000]:
        plt.axvline(x=x, color=component_colors[state])
        x+=1
        
    plt.title("{0} | Log probability of the produced state sequence: {1:.2f}".format(k, decoded_state_sequence[0]))
    plt.yticks([])
    subplot_idx += 1    

print("Rendering...")
plt.subplots_adjust(hspace=0.5)
plt.savefig('output/hmm/decoded_hidden_state_sequence.png')
fig.close()

TODO: How aligned are the state changes between modalities? Cramer's V test?

# Visualize fMRI/EEG Connectome Dynamics

In [ ]:
# for subject_id in ALL_SUBJECT_IDS:
#     for trial_id in ALL_TRIAL_IDS:
        
#         # Attempt to load all connectome types
#         connectomes = load_all_connectome_types(subject_id, trial_id,
#                                                atlas='desikan', 
#                                                seconds_used_to_compute_fmri_connectome=60,
#                                                exclude_bad_fmri_frames=True,
#                                                filter_artifact_timepoints=True)
        
#         if connectomes is None:
#             continue

#         # Plot connectomes through time
#         for t in range(0, connectomes['fmri'].shape[0]):
            
#             # Create figure and set title
#             fig = plt.figure(figsize=(30, 35))
#             fig.suptitle('Subject: "{0}" | Trial: {1} | Time: {2}'.format(subject_id, trial_id, t), fontsize=50)
            
#             # Plot connectomes
#             subplot_idx = 1
#             for connectome_id, connectome in connectomes.items():

#                 ax = fig.add_subplot(len(connectomes), 2, subplot_idx)
#                 plotting.plot_connectome(connectome[t], desikan_atlas_coordinates(), title='{0} Connectome'.format(connectome_id),
#                                          edge_threshold='95%', node_size=20, colorbar=True, axes=ax)
#                 subplot_idx += 1
            
#                 ax = fig.add_subplot(len(connectomes), 2, subplot_idx)
#                 plotting.plot_matrix(connectome[t], vmin=-1., vmax=1., colorbar=True, axes=ax)
#                 subplot_idx += 1
    
#             plt.savefig('output/connectomes_through_time/subject={0}_trial={1}_t={2}.png'.format(subject_id, trial_id, t))